In [55]:
import json
import numpy as np
import pandas as pd
import datetime

In [56]:
c_year = datetime.datetime.now().year

In [57]:
with open("../data/clinical.cases_selection.2019-10-30.json") as f:
    j = json.load(f)

In [58]:
dd = []
ignored = []
for jj in j:
    ddx = {}
    if "demographic" not in jj:
        ignored.append(jj)
        continue
    ddx["days_to_death"] = jj["demographic"].get("days_to_death", None)
    ddx["prefix"] = jj["demographic"]["submitter_id"].split("_")[0]
    ddx["primary_diagnosis"] = jj["diagnoses"][0]["primary_diagnosis"]
    ddx["diagnosis_year"] = jj["diagnoses"][0]["year_of_diagnosis"]
    dd.append(ddx)
df = pd.DataFrame.from_records(dd)

In [59]:
df = (
    df.assign(time_since_diagnosis_aprox=lambda _df: (c_year - _df.diagnosis_year) * 365)
)
df.time_since_diagnosis_aprox.value_counts()

2920.0     1579
3285.0     1362
2555.0     1347
3650.0      998
4015.0      901
2190.0      873
4380.0      697
4745.0      601
5110.0      482
5475.0      392
5840.0      289
6205.0      254
6570.0      207
6935.0      168
7300.0      125
7665.0      100
8030.0       83
8395.0       68
8760.0       60
9125.0       50
9490.0       28
9855.0       26
10585.0       7
10220.0       7
11680.0       6
10950.0       5
11315.0       3
12775.0       2
14600.0       1
13505.0       1
12045.0       1
12410.0       1
14965.0       1
Name: time_since_diagnosis_aprox, dtype: int64

In [60]:
# check out all these people without a diagnosis year
df.time_since_diagnosis_aprox.isna().sum()

127

In [61]:
# these are useless
(df.time_since_diagnosis_aprox.isna() & df.days_to_death.isna()).sum()

105

In [62]:
df["problematic"] = (df.time_since_diagnosis_aprox.isna() & df.days_to_death.isna())

In [63]:
df.head()

,days_to_death,prefix,primary_diagnosis,diagnosis_year,time_since_diagnosis_aprox,problematic
0,NaN,TCGA-A3-3336,"Clear cell adenocarcinoma, NOS",2007.0,4380.0,False
1,1404.0,TCGA-CZ-4854,"Clear cell adenocarcinoma, NOS",2005.0,5110.0,False
2,NaN,TCGA-B0-5698,"Clear cell adenocarcinoma, NOS",2005.0,5110.0,False
3,722.0,TCGA-CZ-5454,"Clear cell adenocarcinoma, NOS",2006.0,4745.0,False
4,313.0,TCGA-B0-4844,"Clear cell adenocarcinoma, NOS",2004.0,5475.0,False


In [67]:
df.to_csv("../data/histology_image_annotations.csv", index=False)

In [68]:
n_diagnoses = df[df.primary_diagnosis != "Not Reported"].primary_diagnosis.value_counts().sort_values(ascending=False)
list(n_diagnoses[:10].index)

['Adenocarcinoma, NOS',
 'Squamous cell carcinoma, NOS',
 'Infiltrating duct carcinoma, NOS',
 'Papillary adenocarcinoma, NOS',
 'Glioblastoma',
 'Serous cystadenocarcinoma, NOS',
 'Clear cell adenocarcinoma, NOS',
 'Malignant melanoma, NOS',
 'Hepatocellular carcinoma, NOS',
 'Transitional cell carcinoma']

In [69]:
len(df)

10852